In [104]:
mask = 0x1
i = 0
comm_size=3
rank=1

while (mask < comm_size):
    dst = rank ^ mask
    
    dst_tree_root = dst >> i
    dst_tree_root <<= i
    
    my_tree_root = rank >> i
    my_tree_root <<= i
    if dst<comm_size:
        print('A', i, dst)
    
    if (dst_tree_root + mask > comm_size):
        nprocs_completed = comm_size - my_tree_root - mask
        print('nprocs_completed: ', nprocs_completed)
        j = mask
        k = 0
        while (j):
            j >>= 1
            k += 1
        k -= 1
        tmp_mask = mask >> 1
        
        while (tmp_mask):
            dst = rank ^ tmp_mask
            tree_root = rank >> k
            tree_root <<= k
            if ((dst > rank) and (rank < tree_root + nprocs_completed) and (dst >= tree_root + nprocs_completed)):
                print('{}- Send to {}'.format(i, dst))
            elif ((dst < rank) and (dst < tree_root + nprocs_completed) and (rank >= tree_root + nprocs_completed)):
                print('{}- Receive {} from {}'.format(i, comm_size - (my_tree_root + mask), dst))
            
            tmp_mask >>= 1
            k -= 1
    
    mask <<= 1
    i+=1

A 0 0
nprocs_completed:  1
1- Receive 1 from 0


In [7]:
n = 3
p = 3

size = n*p


# Inter-Node
for i in range(size):
    left_index = (size + i - p) % size
    right_index = (size + i + p) % size
    
    send_index = i
    recv_index = left_index
    
    for j in range(n-1):
        send_rank = send_index
        recv_rank = recv_index
        
        #print('{} is going to send from {} to {} and receive from {} at {}'.format(i, send_rank, right_index, left_index, recv_rank))
    
        send_index = recv_index
        recv_index = (size + recv_index - p) % size
        
        
# Intra_Node
nodes = [0 for x in range(size)]
for i in range(n):
    for j in range(p):
        nodes[i*p+j] = i
        

for i in range(size):
    right_index = i + 1
    if(right_index >= size):
        right_index -= p
    elif(nodes[i] != nodes[right_index]):
        right_index -= p
        
    left_index = i - 1
    if(left_index < 0):
        left_index += p
    elif(nodes[i] != nodes[right_index]):
        left_index += p
    
    send_index = i
    recv_index = left_index
    
    for j in range(1, p):
        for k in range(n):
            send_rank = (size + send_index - k*p) % size
            recv_rank = (size + recv_index - k*p) % size
            print('{} is going to send from {} to {} and receive from {} at {}'.format(i, send_rank, right_index, left_index, recv_rank))
        send_index = recv_index
        recv_index = recv_index - 1
        if recv_index < 0:
            recv_index += p
        elif(nodes[i] != nodes[recv_index]):
            recv_index += p
            

    

0 is going to send from 0 to 1 and receive from 2 at 2
0 is going to send from 6 to 1 and receive from 2 at 8
0 is going to send from 3 to 1 and receive from 2 at 5
0 is going to send from 2 to 1 and receive from 2 at 1
0 is going to send from 8 to 1 and receive from 2 at 7
0 is going to send from 5 to 1 and receive from 2 at 4
1 is going to send from 1 to 2 and receive from 0 at 0
1 is going to send from 7 to 2 and receive from 0 at 6
1 is going to send from 4 to 2 and receive from 0 at 3
1 is going to send from 0 to 2 and receive from 0 at 2
1 is going to send from 6 to 2 and receive from 0 at 8
1 is going to send from 3 to 2 and receive from 0 at 5
2 is going to send from 2 to 0 and receive from 1 at 1
2 is going to send from 8 to 0 and receive from 1 at 7
2 is going to send from 5 to 0 and receive from 1 at 4
2 is going to send from 1 to 0 and receive from 1 at 0
2 is going to send from 7 to 0 and receive from 1 at 6
2 is going to send from 4 to 0 and receive from 1 at 3
3 is going

# ML RD

In [4]:


n, p = 5, 8

comm_size = n * p


data = [[-1 for x in range(comm_size)] for y in range(comm_size)]
tmp_data = [[-2 for x in range(n)] for y in range(comm_size)]

for i in range(comm_size):
    data[i][i]=i
    tmp_data[i][i//p] = i


curr_cnt = 1
i = 0
mask = 0x1
while (mask < comm_size):
    for rank in range(comm_size):
        my_node = rank//p
        
        dst_node = my_node ^ mask
        dst = rank + (dst_node - my_node) * p


        dst_tree_root = dst_node >> i
        dst_tree_root <<= i



        my_tree_root = my_node >> i
        my_tree_root <<= i

    #    print(dst_node, dst)

        if dst<comm_size:
            #print(rank, dst, dst_node, dst_tree_root, my_tree_root)

            tmp_data[rank][dst_tree_root:min(dst_tree_root+curr_cnt, n)] = tmp_data[dst][dst_tree_root:min(dst_tree_root+curr_cnt, n)]
            #tmp_data[dst][my_tree_root:min(my_tree_root+curr_cnt, comm_size)] = tmp_data[rank][my_tree_root:min(my_tree_root+curr_cnt, comm_size)]
        
            for s in range(min(curr_cnt, n-dst_tree_root)):
                idx = rank + (dst_tree_root+s-my_node) * p
                data[rank][idx] = tmp_data[rank][dst_tree_root+s] 
        if (dst_tree_root + mask > n):
            nprocs_completed = n - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset=my_tree_root + mask
            #print(rank, my_tree_root, mask, offset)
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            tmp_mask = mask >> 1
            
            while (tmp_mask):
                dst_node = my_node ^ tmp_mask
                dst = rank + (dst_node - my_node) * p
        
                tree_root = my_node >> k
                tree_root <<= k
#                 if ((dst_node > my_node) and (my_node < tree_root + nprocs_completed) and (dst_node >= tree_root + nprocs_completed)):
#                     tmp_data[dst][offset:min(offset+curr_cnt, n)] = tmp_data[rank][offset:min(offset+curr_cnt, n)]
#                     for s in range(curr_cnt):
#                         idx = rank + (offset+s-my_node) * p
#                         if idx<comm_size:
#                             data[dst][idx] = tmp_data[dst][offset+s] 
                if ((dst_node < my_node) and (dst_node < tree_root + nprocs_completed) and (my_node >= tree_root + nprocs_completed)):
                    tmp_data[rank][offset:min(offset+curr_cnt, n)] = tmp_data[dst][offset:min(offset+curr_cnt, n)]
                    for s in range(min(curr_cnt, n-offset)):
                        idx = rank + (offset+s-my_node) * p
                        #print(rank, idx)
#                         if idx<comm_size:
                        data[rank][idx] = tmp_data[rank][offset+s] 

                tmp_mask >>= 1
                k -= 1
    curr_cnt *= 2
    mask <<= 1
    i+=1
    
# for d in range(len(data)):
#     print(data[d], d)
print('================= Intra-Node =================')
mask = 0x1
i = 0
curr_cnt = 1

shmem_ranks = [x%p for x in range(comm_size)]

while (mask < p):
    for rank in range(comm_size):
        sh_rank = shmem_ranks[rank]
        sh_dst = sh_rank ^ mask
        my_node = rank//p
        dst = my_node*p + sh_dst

        dst_tree_root = sh_dst >> i
        dst_tree_root <<= i

        
        my_tree_root = sh_rank >> i
        my_tree_root <<= i
        if dst<(my_node+1)*p and dst>=(my_node)*p:
            
            for s in range(n):
                #print(rank, (dst_tree_root+s*p)%comm_size, (my_tree_root+s*p)%comm_size)
                for t in range(min(curr_cnt, p - dst_tree_root)):
                    my_idx = (my_tree_root + s*p) % comm_size +t
                    dst_idx =  (dst_tree_root + s*p) % comm_size + t
#                     if my_idx < comm_size:
#                         print(rank, ' sends to ', dst, ' from ', my_idx)
#                         data[dst][my_idx] = data[rank][my_idx]
#                         if(rank==3 and dst ==5):
#                             print('Yes', data[dst])
#                             print('No', data[rank])
                    if dst_idx < comm_size:
#                         print(rank, ' receives from ', dst, ' at ', dst_idx, dst_tree_root)
                        data[rank][dst_idx] = data[dst][dst_idx]
#                 for tt in range(((my_node + 1) * p - dst_tree_root)):
#                     dst_idx =  (dst_tree_root+s*p)%comm_size + tt
#                     if dst_idx < comm_size:
#                         print(rank, ' receives from ', dst, ' at ', dst_idx)
#                         data[rank][dst_idx] = data[dst][dst_idx]
                    

#         print('----')
        if (dst_tree_root + mask > p):
            nprocs_completed = p - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset = my_tree_root + mask
            
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            
            tmp_mask = mask >> 1

            while (tmp_mask):
                sh_dst = sh_rank ^ tmp_mask
                dst = my_node*p + sh_dst
                tree_root = sh_rank >> k
                tree_root <<= k
                
#                 print(rank, dst, sh_rank, sh_dst, tree_root, nprocs_completed) 
#                 print(rank, ' is here with dst = ', dst) 
                if ((sh_dst > sh_rank) and (sh_rank < tree_root + nprocs_completed) and (sh_dst >= tree_root + nprocs_completed)):
#                     print('S ', rank, ' is here with dst = ', dst) 
                    for s in range(n):
                        for t in range(min(curr_cnt, p-(my_tree_root + mask))):
#                             print(rank, ' Sends to ', dst, ' from ', idx)
                            idx =  (offset+s*p)%comm_size + t
#                             if idx < comm_size:
#                                 data[dst][idx] = data[rank][idx]
                
                elif ((sh_dst < sh_rank) and (sh_dst < tree_root + nprocs_completed) and (sh_rank >= tree_root + nprocs_completed)):
#                     print(rank, ' is here with dst = ', dst) 
                    for s in range(n):
                        for t in range(min(curr_cnt, p-(my_tree_root + mask))):
                            idx =  (offset+s*p)%comm_size + t
                            if idx < comm_size:
#                                 print(rank, ' recievs from ', dst, ' at ', idx)
                                data[rank][idx] = data[dst][idx]
                
                tmp_mask >>= 1
                k -= 1
    
#     for d in range(len(data)):
#         print(data[d], d)
#     print('-----------------------------------------------')
    mask <<= 1
    i+=1
    curr_cnt *= 2
    
# for d in range(len(data)):
#     print(data[d], d)

================= Intra-Node =================


In [ ]:
n, p = 4, 4

comm_size = n * p
recvcount = 16


data = [[-1 for x in range(comm_size*recvcount)] for y in range(comm_size)]
tmp_data = [[-2 for x in range(n*recvcount)] for y in range(comm_size)]

for i in range(comm_size):
    for j in range(recvcount):
        data[i][i*recvcount + j]=i
        tmp_data[i][(i//p)*recvcount + j] = i



curr_cnt = 1
i = 0
mask = 0x1
while (mask < comm_size):
    for rank in range(comm_size):
        my_node = rank//p
        
        dst_node = my_node ^ mask
        dst = rank + (dst_node - my_node) * p


        dst_tree_root = dst_node >> i
        dst_tree_root <<= i



        my_tree_root = my_node >> i
        my_tree_root <<= i

    #    print(dst_node, dst)

        if dst<comm_size:
            #print(rank, dst, dst_node, dst_tree_root, my_tree_root)

            tmp_data[rank][dst_tree_root*recvcount:min(dst_tree_root+curr_cnt, n)*recvcount] = tmp_data[dst][dst_tree_root*recvcount:min(dst_tree_root+curr_cnt, n)*recvcount]
            #tmp_data[dst][my_tree_root:min(my_tree_root+curr_cnt, comm_size)] = tmp_data[rank][my_tree_root:min(my_tree_root+curr_cnt, comm_size)]

            for s in range(min(curr_cnt, n-dst_tree_root)):
                idx = rank + (dst_tree_root+s-my_node) * p
                if idx<comm_size:
                    data[rank][idx*recvcount:(idx+1)*recvcount] = tmp_data[rank][(dst_tree_root+s)*recvcount:(dst_tree_root+s+1)*recvcount]
                    
                    
        if (dst_tree_root + mask > n):
            nprocs_completed = n - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset=my_tree_root + mask
            #print(rank, my_tree_root, mask, offset)
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            tmp_mask = mask >> 1
            
            while (tmp_mask):
                dst_node = my_node ^ tmp_mask
                dst = rank + (dst_node - my_node) * p
        
                tree_root = my_node >> k
                tree_root <<= k
#                 if ((dst_node > my_node) and (my_node < tree_root + nprocs_completed) and (dst_node >= tree_root + nprocs_completed)):
#                     tmp_data[dst][offset:min(offset+curr_cnt, n)] = tmp_data[rank][offset:min(offset+curr_cnt, n)]
#                     for s in range(curr_cnt):
#                         idx = rank + (offset+s-my_node) * p
#                         if idx<comm_size:
#                             data[dst][idx] = tmp_data[dst][offset+s] 
                if ((dst_node < my_node) and (dst_node < tree_root + nprocs_completed) and (my_node >= tree_root + nprocs_completed)):
                    tmp_data[rank][offset*recvcount:min(offset+curr_cnt, n)*recvcount] = tmp_data[dst][offset*recvcount:min(offset+curr_cnt, n)*recvcount]
                    
                    for s in range(min(curr_cnt, n-offset)):
                        idx = rank + (offset+s-my_node) * p
                        #idx= rank + offset
                        #print(rank, idx)
                        if idx<comm_size:
                            data[rank][idx*recvcount:(idx+1)*recvcount] = tmp_data[rank][(offset+s)*recvcount:(offset+s+1)*recvcount] 

                tmp_mask >>= 1
                k -= 1
    curr_cnt *= 2
    mask <<= 1
    i+=1
    

print('================= Intra-Node =================')
mask = 0x1
i = 0
curr_cnt = 1

shmem_ranks = [x%p for x in range(comm_size)]

while (mask < p):
    for rank in range(comm_size):
        sh_rank = shmem_ranks[rank]
        sh_dst = sh_rank ^ mask
        my_node = rank//p
        dst = my_node*p + sh_dst

        dst_tree_root = sh_dst >> i
        dst_tree_root <<= i

        
        my_tree_root = sh_rank >> i
        my_tree_root <<= i
        if dst<(my_node+1)*p and dst>=(my_node)*p:
            
            for s in range(n):
                #print(rank, (dst_tree_root+s*p)%comm_size, (my_tree_root+s*p)%comm_size)
                for t in range(min(curr_cnt, p - dst_tree_root)):
                    my_idx = (my_tree_root + s*p) % comm_size +t
                    dst_idx =  (dst_tree_root + s*p) % comm_size + t
#                     if my_idx < comm_size:
#                         print(rank, ' sends to ', dst, ' from ', my_idx)
#                         data[dst][my_idx] = data[rank][my_idx]
#                         if(rank==3 and dst ==5):
#                             print('Yes', data[dst])
#                             print('No', data[rank])
                    if dst_idx < comm_size:
                        print(rank, ' receives from ', dst, ' at ', dst_idx, dst_tree_root)
                        data[rank][dst_idx*recvcount:(dst_idx+1)*recvcount] = data[dst][dst_idx*recvcount:(dst_idx+1)*recvcount]
#                 for tt in range(((my_node + 1) * p - dst_tree_root)):
#                     dst_idx =  (dst_tree_root+s*p)%comm_size + tt
#                     if dst_idx < comm_size:
#                         print(rank, ' receives from ', dst, ' at ', dst_idx)
#                         data[rank][dst_idx] = data[dst][dst_idx]
                    

#         print('----')
        if (dst_tree_root + mask > p):
            nprocs_completed = p - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset = my_tree_root + mask
            
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            
            tmp_mask = mask >> 1

            while (tmp_mask):
                sh_dst = sh_rank ^ tmp_mask
                dst = my_node*p + sh_dst
                tree_root = sh_rank >> k
                tree_root <<= k
                
#                 print(rank, dst, sh_rank, sh_dst, tree_root, nprocs_completed) 
#                 print(rank, ' is here with dst = ', dst) 
                if ((sh_dst > sh_rank) and (sh_rank < tree_root + nprocs_completed) and (sh_dst >= tree_root + nprocs_completed)):
#                     print('S ', rank, ' is here with dst = ', dst) 
                    for s in range(n):
                        for t in range(min(curr_cnt, p-(my_tree_root + mask))):
#                             print(rank, ' Sends to ', dst, ' from ', idx)
                            idx =  (offset+s*p)%comm_size + t
                            if idx < comm_size:
                                print(rank, 'is going to send from ', idx, ' to ',dst)
#                                 data[dst][idx] = data[rank][idx]
                
                elif ((sh_dst < sh_rank) and (sh_dst < tree_root + nprocs_completed) and (sh_rank >= tree_root + nprocs_completed)):
#                     print(rank, ' is here with dst = ', dst) 
                    for s in range(n):
                        for t in range(min(curr_cnt, p-(my_tree_root + mask))):
                            idx =  (offset+s*p)%comm_size + t
                            if idx < comm_size:
#                                 print(rank, ' recievs from ', dst, ' at ', idx)
                                print(rank, 'is going to recv at ', idx, ' from ', dst)
                                data[rank][idx*recvcount:(idx+1)*recvcount] = data[dst][idx*recvcount:(idx+1)*recvcount]
                
                tmp_mask >>= 1
                k -= 1
    
#     for d in range(len(data)):
#         print(data[d], d)
#     print('-----------------------------------------------')
    mask <<= 1
    i+=1
    curr_cnt *= 2
    
# for d in range(len(data)):
#     print(data[d], d)

# for d in data:
#     for i in range(comm_size):
#         for j in range(recvcount):
#             if d[i*recvcount+j] != i:
#                 print(':-(')

print('Finito')



In [1]:
l = [6, 1, 2, 8, 5, 7, 0, 4, 3]
p=3
nodes = [-1 for x in range(len(l))]

for x in l:
    nodes[x] = l.index(x)//p 

print(nodes)

[2, 0, 0, 2, 2, 1, 0, 1, 1]


## Non-Blocked

In [11]:
from random import shuffle

n, p = 8, 4

comm_size = n * p
recvcount = 16
rank_list = [x for x in range(comm_size)]
nodes = [-1 for x in range(comm_size)]
# shuffle(rank_list)
# for i in range(n):
#     for j in range(p):
#         rank_list[i*p+j] = i+j*n

for x in rank_list:
    nodes[x] = rank_list.index(x)//p 
# rank_list = [6, 1, 2, 8, 5, 7, 0, 4, 3]


# print(rank_list, nodes)

data = [[-1 for x in range(comm_size*recvcount)] for y in range(comm_size)]
result = [[-1 for x in range(comm_size*recvcount)] for y in range(comm_size)]
tmp_data = [[-2 for x in range(n*recvcount)] for y in range(comm_size)]

for i in range(comm_size):
    for j in range(recvcount):
        data[rank_list[i]][i*recvcount + j]=rank_list[i]
        tmp_data[rank_list[i]][(nodes[rank_list[i]])*recvcount + j] = rank_list[i]

# for d in range(len(data)):
#     print(data[d], d)

# for d in range(len(tmp_data)):
#     print(tmp_data[d], d)

curr_cnt = 1
i = 0
mask = 0x1
while (mask < comm_size):
    for rank_idx in range(comm_size):
        rank = rank_list[rank_idx]
        my_node = rank_idx//p
        
        dst_node = my_node ^ mask
        dst_idx = rank_idx + (dst_node - my_node) * p


        dst_tree_root = dst_node >> i
        dst_tree_root <<= i



        my_tree_root = my_node >> i
        my_tree_root <<= i

    #    print(dst_node, dst)

        if dst_idx<comm_size:
            dst= rank_list[dst_idx]
#             print(rank, ' recvs from ', dst, ' at ', dst_tree_root)
            tmp_data[rank][dst_tree_root*recvcount:min(dst_tree_root+curr_cnt, n)*recvcount] = tmp_data[dst][dst_tree_root*recvcount:min(dst_tree_root+curr_cnt, n)*recvcount]
            #tmp_data[dst][my_tree_root:min(my_tree_root+curr_cnt, comm_size)] = tmp_data[rank][my_tree_root:min(my_tree_root+curr_cnt, comm_size)]

            for s in range(min(curr_cnt, n-dst_tree_root)):
                idx = rank_idx + (dst_tree_root+s-my_node) * p
                if idx<comm_size:
                    data[rank][idx*recvcount:(idx+1)*recvcount] = tmp_data[rank][(dst_tree_root+s)*recvcount:(dst_tree_root+s+1)*recvcount]
                    
                    
        if (dst_tree_root + mask > n):
            nprocs_completed = n - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset=my_tree_root + mask
            #print(rank, my_tree_root, mask, offset)
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            tmp_mask = mask >> 1
            
            while (tmp_mask):
                dst_node = my_node ^ tmp_mask
                dst_idx = rank_idx + (dst_node - my_node) * p
        
                tree_root = my_node >> k
                tree_root <<= k
#                 if ((dst_node > my_node) and (my_node < tree_root + nprocs_completed) and (dst_node >= tree_root + nprocs_completed)):
#                     tmp_data[dst][offset:min(offset+curr_cnt, n)] = tmp_data[rank][offset:min(offset+curr_cnt, n)]
#                     for s in range(curr_cnt):
#                         idx = rank + (offset+s-my_node) * p
#                         if idx<comm_size:
#                             data[dst][idx] = tmp_data[dst][offset+s] 
                if ((dst_node < my_node) and (dst_node < tree_root + nprocs_completed) and (my_node >= tree_root + nprocs_completed)):
                    dst = rank_list[dst_idx]
                    tmp_data[rank][offset*recvcount:min(offset+curr_cnt, n)*recvcount] = tmp_data[dst][offset*recvcount:min(offset+curr_cnt, n)*recvcount]
                    
                    for s in range(min(curr_cnt, n-offset)):
                        idx = rank_idx + (offset+s-my_node) * p
                        #idx= rank + offset
                        #print(rank, idx)
                        if idx<comm_size:
                            data[rank][idx*recvcount:(idx+1)*recvcount] = tmp_data[rank][(offset+s)*recvcount:(offset+s+1)*recvcount] 

                tmp_mask >>= 1
                k -= 1
    curr_cnt *= 2
    mask <<= 1
    i+=1

# for d in range(len(tmp_data)):
#      print(tmp_data[d], d)

print('================= Intra-Node =================')
mask = 0x1
i = 0
curr_cnt = 1

shmem_ranks = [x%p for x in range(comm_size)]

while (mask < p):
    for rank_idx in range(comm_size):
        sh_rank = shmem_ranks[rank_idx]
        sh_dst = sh_rank ^ mask
        my_node = rank_idx//p
        dst_idx = my_node*p + sh_dst

        dst_tree_root = sh_dst >> i
        dst_tree_root <<= i

        
        my_tree_root = sh_rank >> i
        my_tree_root <<= i
        if dst_idx<(my_node+1)*p and dst_idx>=(my_node)*p:
            rank, dst = rank_list[rank_idx], rank_list[dst_idx]
            for s in range(n):
                #print(rank, (dst_tree_root+s*p)%comm_size, (my_tree_root+s*p)%comm_size)
                for t in range(min(curr_cnt, p - dst_tree_root)):
                    send_idx = (my_tree_root + s*p) % comm_size +t
                    recv_idx =  (dst_tree_root + s*p) % comm_size + t
                    if send_idx < comm_size:
#                         print(rank, ' sends to ', dst, ' from ', send_idx)
#                         data[dst][my_idx] = data[rank][my_idx]
#                         if(rank==3 and dst ==5):
#                             print('Yes', data[dst])
#                             print('No', data[rank])
#                     if recv_idx < comm_size:
                        
#                         print(rank, ' receives from ', dst, ' at ', recv_idx)
                        data[rank][recv_idx*recvcount:(recv_idx+1)*recvcount] = data[dst][recv_idx*recvcount:(recv_idx+1)*recvcount]
                    

#         print('--------------------------------')
        if (dst_tree_root + mask > p):
            nprocs_completed = p - my_tree_root - mask
            #print('nprocs_completed: ', nprocs_completed)
            offset = my_tree_root + mask
            
            j = mask
            k = 0
            while (j):
                j >>= 1
                k += 1
            k -= 1
            
            tmp_mask = mask >> 1

            while (tmp_mask):
                sh_dst = sh_rank ^ tmp_mask
                dst_idx = my_node*p + sh_dst
                tree_root = sh_rank >> k
                tree_root <<= k
                
#                 print(rank, dst, sh_rank, sh_dst, tree_root, nprocs_completed) 
#                 print(rank, ' is here with dst = ', dst) 
                if ((sh_dst > sh_rank) and (sh_rank < tree_root + nprocs_completed) and (sh_dst >= tree_root + nprocs_completed)):
#                     print('S ', rank, ' is here with dst = ', dst) 
                    rank, dst = rank_list[rank_idx], rank_list[dst_idx]
                    for s in range(n):
                        for t in range(min(curr_cnt, p-(my_tree_root + mask))):
#                             print(rank, ' Sends to ', dst, ' from ', idx)
                            idx =  (offset+s*p)%comm_size + t
#                             if idx < comm_size:
#                                 print(rank, ' sends from ', idx, ' to ',dst)
#                                 data[dst][idx] = data[rank][idx]
                
                elif ((sh_dst < sh_rank) and (sh_dst < tree_root + nprocs_completed) and (sh_rank >= tree_root + nprocs_completed)):
                    rank, dst = rank_list[rank_idx], rank_list[dst_idx]
#                                 print(rank, ' is here with dst = ', dst) 
                    for s in range(n):
                        for t in range(min(curr_cnt, p-(my_tree_root + mask))):
                            idx =  (offset+s*p)%comm_size + t
                            if idx < comm_size:
#                                 print(rank, ' recievs from ', dst, ' at ', idx)
#                                 print(rank, 'is going to recv at ', idx, ' from ', dst)
                                data[rank][idx*recvcount:(idx+1)*recvcount] = data[dst][idx*recvcount:(idx+1)*recvcount]
                
                tmp_mask >>= 1
                k -= 1
    
#     for d in range(len(data)):
#         print(data[d], d)
#     print('-----------------------------------------------')
    mask <<= 1
    i+=1
    curr_cnt *= 2
    
# for d in range(len(data)):
#     print(data[d], d)

    
print('--------- Rotation ----------')
for rank_idx in range(comm_size):
    rank = rank_list[rank_idx]
    for i in range(comm_size):
        dst = rank_list[i]
        result[rank][dst*recvcount:(dst+1)*recvcount] = data[i][i*recvcount:(i+1)*recvcount]
    
    
for d in data:
    for i in range(comm_size):
        for j in range(recvcount):
            if d[i*recvcount+j] != rank_list[i]:
                print(':-(')

for d in result:
    for i in range(comm_size):
        for j in range(recvcount):
            if d[i*recvcount+j] != i:
                print(';-[[')                
                
                
print('Finito')

# for d in range(len(result)):
#     print(result[d], d)


================= Intra-Node =================
--------- Rotation ----------
Finito


# Blocked or NonBlocked?

In [1]:
ranks = [x for x in range(12)]
nodes = [3, 3, 3, 1, 1, 1, 2, 2, 0, 0, 0,0]

local_sizes=dict()
for x in nodes:
    local_sizes[x]=local_sizes.get(x, 0)+1

    
tmp, prev_node, is_globally_blocked=-1, -1, 1
for i in range(len(ranks)):
    node = nodes[i]
    if prev_node==-1:
        tmp=local_sizes[node]-1
        prev_node=node
    elif tmp>0:
        if node==prev_node:
            tmp-=1
            if tmp==0:
                prev_node=-1
        else:
            is_globally_blocked=0
            break
            
if(is_globally_blocked):
    print('Is Globally Blocked', is_globally_blocked)
else:
    print('Is Not Globally Blocked', is_globally_blocked)

Is Globally Blocked 1


# Non-Overlapped Indices

In [5]:
n, p = 8, 4

comm_size = n * p
curr_cnt = 1

rank_list = [x for x in range(comm_size)]
nodes = [-1 for x in range(comm_size)]


for x in rank_list:
    try:
        nodes[x] = rank_list.index(x)//p 
    except:
        print('Khar')
# rank_list = [6, 1, 2, 8, 5, 7, 0, 4, 3]


i = 0
mask = 0x1
recently_received = 1
while (mask < comm_size):
    for rank_idx in range(comm_size):
        rank = rank_list[rank_idx]
        try:
            my_node = rank_idx//p
        except:
            print('Sag')
        dst_node = my_node ^ mask
        dst_idx = rank_idx + (dst_node - my_node) * p


        dst_tree_root = dst_node >> i
        dst_tree_root <<= i



        my_tree_root = my_node >> i
        my_tree_root <<= i

    #    print(dst_node, dst)

        if dst_idx<comm_size:
            dst= rank_list[dst_idx]
            recv_offset = dst_tree_root
            for f in range(recently_received):
                enc_in = recv_offset + f
                out_index = rank_idx + (dst_tree_root+f-my_node)*p
                dec_out = rank_list[out_index]
                
#                 print('{} decrypts from {} to {}'.format(rank, enc_in, dec_out))
    recently_received*=2
    mask <<= 1
    i+=1


# Dynamic ShMem

In [18]:
n = 8
p = 16
shmem_leaders = 2
workload = p // shmem_leaders
my_node = 0
size  = 1

for local_rank in range(p):
    for i in range(n):
        if(i!= my_node):
            for l_idx in range(shmem_leaders):
                if ((i+1)*(l_idx+1))%p == local_rank:
                    t = workload 
                    if l_idx == shmem_leaders-1:
                        t += (p-(shmem_leaders * workload))
                    idx = i*p + l_idx*workload
                    next = idx * size + (i *shmem_leaders + l_idx)*(16+12)
                    dest = idx * size
                    
                    print('{} is going to decrypt {} from {} to {}'.format(local_rank, t, next, dest))
                

0 is going to decrypt 8 from 540 to 120
2 is going to decrypt 8 from 72 to 16
3 is going to decrypt 8 from 144 to 32
4 is going to decrypt 8 from 108 to 24
4 is going to decrypt 8 from 216 to 48
5 is going to decrypt 8 from 288 to 64
6 is going to decrypt 8 from 180 to 40
6 is going to decrypt 8 from 360 to 80
7 is going to decrypt 8 from 432 to 96
8 is going to decrypt 8 from 252 to 56
8 is going to decrypt 8 from 504 to 112
10 is going to decrypt 8 from 324 to 72
12 is going to decrypt 8 from 396 to 88
14 is going to decrypt 8 from 468 to 104


In [ ]:
2 is going to decrypt from 4194332 to 4194304
3 is going to decrypt from 8388664 to 8388608
4 is going to decrypt from 12582996 to 12582912